### Interface

Har brukervenlig design. Ulike type estimatorer (prediktor/transformers) har samme interface slik at jeg kan behandle de på samme måte og dytte de inn i meta-estimatorer som pipeline/gridsearch (som igjen har samme interface utad). Kan også lage custom estimators så lengde de oppfører seg på samme måte.

```python
class Estimator(...):
    def __init__(self, hyperparam, ...):

    def fit(self, X, y):   # Fit/model the training data
        ...                # given data X and targets y
        return self
    
    if predictor:
        def predict(self, X):  
            ...                
            return y_pred
    
    elif transformer:
        def transform(self, X):
            ...
            return X_transformed
```

#### Supervised

```python
est = Estimator().fit(X,y)

est.predict(X_new)
est.score(X_new) # accuracy for classification, R^2 for regression
est.transform() # vet ikke... reduserer til features som er viktig? hmhm

# Classification
est.predict_proba(X_new)
est.decision_function(X_new) # mål på usikkerhet som ikke er sannsynlighet ...
```

#### Unsupervised

Har lignende interface ... Litt usikker på om det er noe poeng med train test split på unsupervised.
```python
est.transform(X_new) # bruke parameter den har lært fra fit til å transformere..
est.fit_transform(X) # hvis kjøre fit og transform på samme
est.predict(X) # labels for cluster
est.predict_proba(X) # sannsynlighet hvis soft boundary (eks. gmm)
est.score(X) # noe likelihood greier
```

### Base

Kan lage egne estimatorer som følger API. Mine custom class må inherite fra `BaseEstimator`. Har implementasjon av get_params og set_params slik at de fungerer i med GridSearch og Pipeline

Har mange Mixin classer som jeg ikke skjønner hva er... Mixin er mer generelt begrep; ikke bare noe i sklearn.

- RegressorMixin
- TransformerMixin
- mm...

#### Lage custom estimators

Inheriter fra TransformerMixin for å få fit_transform til å bli definert ut fra fit og transforn.

Eksempel som avgrenser til subset av k mest viktige features (som bestemt av feks. feature importance i random forest). Bruker dette til å initialisere objekt og har dessuten hyperparameter k. Kan få dette inn i gridsearch som del av pipeline uten at jeg trenger å finne feature importance i hvert steg siden kun varierer k.

```python
class TopFeatureSelector(BaseEstimator, TransformerMixin):
    def __init__(self, feature_importances, k):
        self.feature_importances = feature_importances
        self.k = k
    def fit(self, X, y=None):
        self.feature_indices_ = indices_of_top_k(self.feature_importances, self.k)
        return self
    def transform(self, X):
        return X[:, self.feature_indices_]
```

### Preprocessing

#### Skalering av data

Mange av algoritmene bruker mål på avstand mellom observasjonene i inputrommet. De numeriske verdiene vi observerer i datasettet avhenger av måleenhet og forskjeller langs ulike kolonner er ikke nødvendigvis sammenlignbare. Algoritmene gir bedre resultat dersom vi forsøker å konvertere til en felles skala.

For numeriske verdier bruker vi som oftest `StandardScaler` som trekker fra gjennomsnitt og deler på standardavvik for hver av kolonnene slik at de får fordelinger med $\bar{x}=0$ og $std(x)=1$. 

Hvis verdiene er ikke-negative kan vi beholde en sparse representasjon ved å bruke `MinMaxScaler` som skalerer verdi inn i intervall [0,1]. Hvis 0-verdi er minste verdi i opprinnelig kolonne så vil de fortsatt være 0 etter transformasjon.

#### Feature engineering

Transformere verdi i kolonne

1. `PowerTransformer`
2. `OneHotEncoder`
3. `PolynomialFeatures`

Kan partisjonere kontinuerlig inputmengde og lage dummy for indikatorer om gitt observasjon er i gitt bin. Litt usikker på hvordan jeg kombinerer med original kontinuerlig verdi slik at binsene kommer i tillegg i stedet for å erstatte...

```python
kb = KBinsDiscretizer(n_bins, # antall bins (hyperparameter)
                      encode, # sparse som default, kan spesifisere 'onehot-dense'
                      strategy # quantile (lik antall obs), uniform (lik avstand)
                      ).fit(X)
kb.bin_edges_
```

Kan lage polynom transformasjoner. Antallet variabler øker veldig raskt når vi øker antall grader fordi det øker antallet faktorer i kryssproduktene/interaksjonene. I praksis bruker vi neppe degree > 3.

```python
poly = PolynomialFeatures(degree).fit_transform(X) # lærer ikke parametre i fit()..
poly.get_feature_names()
```

### Feature Extraction

`TfidfVectorizer`... Tar tekstdokument som input, lage dict med unike ord, lage vektor som angir antall ganger hvert av de unike ordene fra dokumnetene er i hver dokument. 

tror generelt modulen i hovedsak har verktøy for å jobbe med tekstdata

### Feature Selection

Tenker dette er nyttig hvis jeg har et stort datasett med mange observerte variabler og det ikke er opplagt hvilke som er relevante for å predikere utfallet vi er interessert i.

Kunne prøvd alle kombinasjoner av variabler, med det er $2^k$ som vokser raskt. Trenger bedre strategi for å finne et subset av variabler

### Impute

Algoritmer kan ofte ikke håndtere missing data, men medfører tap av informasjon dersom vi kaster bort kolonner der vi ikke har observert verdi for alle. Løsningen er å *impute*/predikere verdi på det som mangler. 

Har i hovedsak to metoder:
1. SimpleImpute som bruker verdier langs den gitte kolonnen
2. KNNImpute som for hver observasjon bruker subset av observasjoner som har lignende verdier langs andre kolonner og observert på den kolonnen som mangler for gitte observasjonen.

Tror det er litt rom for domain knowledge her. Det er også poeng at det kan være ulike kilder til missingness (sannsynlighet for at missing)
1. Totally at random
2. Avhengig av verdi på annen kolonne
3. Avhengig av verdi på den gitte kolonne

Kan gi biased resultat dersom vi ikke tar hensyn til at det ikke er totally random..

```python
X_imp = KNNImputer(n_neighbours).fit_transform(X)

X_imp = SimpleImputer(strategy=
                      'mean',
                      'median',
                      'most_frequent', # kan også brukes på ikke-numerisk
                      'constant' # må spesfisere verdien
                      ).fit_transform()
```

### Metrics

Har metrics som tar y_pred og y_true og gir score

Har scores som tar estimator som argument. Bruker dette i `GridSearchCV` hvis jeg vil ha andre metric enn accuracy som kriterie for valg av beste hyperparameter. Har make_scorer som er najs wrapper funksjon som konverter metric til scorer.

#### Kategorisk

Har ulike metrics for binær klassifisering:
1. Accuracy (andel predikert i riktig kategori)
2. Precision (andel av predikert positive som faktisk er positiv)
3. Recall/sensitivitet (andel av faktisk positive som predikert positiv)

Kan tenke på det som corona-test. Presisjon ser på andelen som får positivt svar som faktisk er smittet. Recall ser på andelen av de som er smittet som får positivt svar. Det er en tradeoff her som avhenger av treshold i decision_function(). Får å øke sensitiviteten må predikere positive for flere av grensetilfellene, men da får vi flere falske positive og dermed lavere presisjon. Merk også at vi kan få arbitrær høy sensitivitet ved å predikere alle positiv.

I praksis vil vi måtte gjøre en avveining, og den optimale balansen avhenger av kostnadene ved ulike typer feil. Vi kan bruke noen sammendragsmål

1. F1-score
2. Areal under ROC-curve..


$$
\text{F} = 2 \cdot \frac{\text{precision} \cdot \text{recall}}{\text{precision} + \text{recall}}
$$

##### Confusion matrix

TP,FP,TN,FN i binær.

Ulike typer feil i multiclass

```python
plot_confusion_matrix(est, X_test, y_test,
                      normalize=, # [pred, true, all]. Bruker vel mest true, summer over rekke
                      
```

##### Precision-Sensitivity tradeoff

Kan undersøke hvordan det avhenger i treshold til output av predict_proba() eller decision_function(). Makismerer accuracy ved å plassere i kategori med høyest sannsynlighet, men ikke alltid accuracy er målet.

```python
precision, recall, treshold = precision_recall_curve(y_test, clf.decision_function(X_test))

# plotte begge som funksjon av treshold
plt.plot(treshold, precision[:-1]) # vet ikke hvorfor output ikke har samme shape ...
plt.plot(treshold, recall[:-1])

# plotte sensitivitet mot presisjon
plt.plot(recall, precision)
```


##### AUC

```python
fpr, tpr, treshold = roc_curve(y_test, clf.decision_function(X_test))
auc = roc_auc_score(y_test, clf.decision_function(X_test))

plt.plot(fpr, tpr, label=f'auc: {auc}')
plt.legend()
```

- confusion_matrix(y_true, y_pred), kan bruke som input i visualisering
- classification_report()
- precision_recall_curve()
- roc_curve()

Har hjelpefunksjoner for raske plot
```python
plot_precision_recall_curve(pipe,X_test, y_test) # presisjon vs sensitivitet (vil opp til høyre)
plot_roc_curve(pipe,X_test, y_test) # false positive rate vs true positive rate (vil opp til venstre)
```

### Pipeline

Enkapsulere alt arbeid i ett python objekt...Vi har transformers og predictors. Bruker output fra transformers (imputation, skalering, mm) som input i predictor. Kan gjøre dette steg for steg, men det blir fort veldig rotete. Kan derfor i stedet lage pipeline gjennomfører transformeringene steg for steg. 

Også fordel at pre-processing blir del av grid i kryssvalidering i stedet for kun kjøre kryssvalidering i siste steget. Kan velge parametre i preproccessing + mindre skjevhet av at den har lært egenskaper fra andre folds i treningsdata fra før


Kan bruke .predict(), .score() og sånn direkte på pipeline. Kjører da gjennom samme prossess tror jeg slik at vi får test data som tilsvarer... 

```python
def fit(self, X, y):
    X_transformed = X
    for name, estimator in self.steps[:-1]:
        # iterate over all but the final step
        # fit and transform the data
        X_transformed = estimator.fit_transform(X_transformed, y)
    # fit the last step
    self.steps[-1][1].fit(X_transformed, y)
    return self
```

```python
def predict(self, X):
    X_transformed = X
    for step in self.steps[:-1]:
        # iterate over all but the final step
        # transform the data
        X_transformed = step[1].transform(X_transformed)
    # predict using the last step
    return self.steps[-1][1].predict(X_transformed)
```

#### Konstruksjon av pipeline

Kan initialisere `Pipeline` objekt direkte eller bruke `make_pipeline` funksjon. Føler at den andre metoden er bedre siden jeg slipper å eksplisitt navngi stegenene; følger automatisk konvensjon med navn på class i lowercase.

```python
steps = [est1(), est2(),..]
pipe = make_pipeline(*steps, # merk at må bruke instance (object) ikke class
                     memory='cache_folder') # kan lagre transformer-objekt ... usikker på dette
```

Eksempel der vi bruker pipeline i grid search.
```python
steps = [StandardScaler(), PolynomialFeatures(), Ridge()]
pipe = make_pipeline(*steps)
param_grid={'polynomialfeatures__degree': [1, 2, 3],
            'ridge__alpha': np.linspace(0.1, 2, 20)}

grid = GridSearchCV(pipeline, param_grid, cv=5).fit(X_train,y_train)
```

#### Tilgang til steg og deres attributter

Kan få ut predictor-objekt i siste steg med
```python
est = pipe.named_steps['navnpåestimator']
```
dette kan være praktisk siden den har metoder og attributter vil vi ha tak i

Kan sette hyperparametre til pipe med

`navn_på_steg__navn_på_param`

#### Feature union

### Compose

`ColumnTransformer`.. vil gjøre ulike transformasjoner på ulike subsets av kolonner. Kjører parallelt inne i pipeline. Merk at vi må bruke columntransformer først i pipeline fordi den tar DataFrame som input, mens andre estimatorer returner arrays fra .fit_transform()

```python
ct = make_column_transformer((OneHotEncoder(), cat_features), # første arg (transformer, list_of_cols)
                             remainder=StandardScaler()) # hvilken transformasjon på resten.
pipe = make_pipeline(ct, Ridge()) # kombinerer til pipe med prediktor i siste ledd. 
```

I stedet for å lage en eksplisitt liste av kategoriske features kan jeg også bruke `make_column_selector` som konstruerer inndeling ut fra kriterie
```python
ct = make_column_transformer((OneHotEncoder(),
                              make_column_selector(dtype_include=np.object)),
                             (StandardScaler(),
                              make_column_selector(dtype_include=np.number)))
```

### Model Selection

Kan finne cross validate score for gitt algoritme med gitt hyperparameter med 

```python
cross_val_score(estimator, X, y, scoring) # splitter i 5 folds, trener på treningsdata og gir list med score på test
```

I praksis vil vi kjøre cv på grid av modeller. Kan gjøre det manuelt for en gitt 2-fold deling slik:

```python
training_accuracy = []
test_accuracy = []
k = range(1, 11)

for n_neighbors in k:
    # build the model
    clf = KNeighborsClassifier(n_neighbors=n_neighbors).fit(X_train, y_train)
    # record training and test set accuracy
    training_accuracy.append(clf.score(X_train, y_train))
    test_accuracy.append(clf.score(X_test, y_test))
```

1. validation_curve()
2. learning_curve() # undersøke verdi av å legge til mer data (eventuelt se om aktuelt å øke fleksibilitet)

```python
param_grid = np.arange(1,10+1)
train_scores, test_scores = validation_curve(KNeighborsClassifier(),X_train,y_train,'n_neighbors',param_grid)
train_scores_mean = np.mean(train_scores,axis=1)
test_scores_mean = np.mean(test_scores,axis=1) 
```

#### GridSearchCV

GridSearchCV er en *meta-estimator* og har lignende api som vanlig estimator. Når vi caller .fit() gjør den kryssvalidering på grid internt, finner beste hyperparameter og trener deretter modellen på hele treningsdata. Det er denne estimatoren som deretter brukes når vi caller .predict() og .score().

Vi kan undersøke resultatene fra kryssvalideringen med .cv_results_()

```python
GridSearchCV(estimator, # i praksis bruker jeg Pipeline som (meta)estimator
             param_grid, # dict, bruker {navn__param_navn:[vals], ..} til å angi hvilken est i pipe
             scoring, # scorer, string (mapper til objekt) eller eksplisitt callable. Kan ta liste.
                      # VIKTIG: forsøker å maxe score; dersom loss (eg. MSE) må jeg bruke negativ verdi. 
             return_train_score=True # default er False, kan sette True hvis jeg vil plotte..
            )
```

Hvis jeg bare vil ha enkel train/validation split for hvert kombinasjon av hyperparameter (f.eks. hvis jeg har veldig mye data og/eller dårlig tid) kan jeg bruke

```python
single_split_cv = ShuffleSplit(n_splits=1)
grid = GridSearchCV(estimator, param_grid, cv=single_split_cv)
```

##### GridSearch og Pipeline

I praksis bruker vi `Pipeline` som estimator i `GridSearchCV`. Den beste estimatoren er dermed et Pipeline-objekt. Hvis vi vil undersøke egenskaper til prediktor i siste steg, for eksempel for å se på coeffisientene i logistisk regresjon, må vi bruke

```python
grid.best_estimator_.named_steps['logistic_regression'].coef_
```

##### Visualisere gridsearch 

grid.cv_results_ er en dictionary som fort blir veldig stor og uoversiktelig med stor grid. Kan få mye bedre representasjon hvis jeg transformerer det til dataframe. Kan lage plots (type validation curves og sånn) med utgangspunkt i dette.

For hvert punkt i grid har det info om
1. Hvor lang tid det tar
2. Parameterkombinasjon (dict)
3. Verdi av hver parameter
4. Score på hver fold
5. Gjennomsnittlig score og std. avvik.

```python
pd.DataFrame(grid.cv_results_)[['param_ridge__alpha','mean_train_score','mean_test_score']]
```
Hvis en scorer er navn bare `score`, hvis flere scores så bruker navn på scorer i stedet, eks `mean_test_neg_mean_square_error`. 

Tror jeg har lyst til å lage hjelpe-funksjon som plotter resultat.. Litt vanskelig med flere parametre..

#### RandomizedSearchCV

Fordeler med `RandomizedSearchCV`:
1. Slipper å spesifisere eksakt steps i grid (liker ikke arbritrære valg)
2. Kontrollere kompleksitet med n_iter (i stedet for å tenke på størrelse av grid)
3. Robust for irrelevant parameter (Randomisert så hver steg har variasjon i relevante, i stedet for å loope over fixed verdi av andre parametre for hver verdi av irrelevant)

```python
rs = RandomizedSearchSV(pipe,
                        param_distributions, # dict med {'navn':fordeling,..} der fordeling har .rvs()
                        n_iter # spesifisere hvor mange trekk fra fordelingene. Kontrollere lengde på search
                        )
```

#### Utvide til grid search på flere predictors

Kunne kjørt loop på ulik grid for ulik predictor, f.eks:

```python
preprocess_steps = [ct, KNNImputer()]

predictors = [KNeighborsClassifier(), LinearSVC(tol=1)]
param_grids = [{'kneighborsclassifier__n_neighbors':np.arange(1,30+1,5)},
               {'linearsvc__C':np.linspace(0.1,3,10)}
              ]

grids = []
for i in range(len(predictors)):
    steps = preprocess_steps+[predictors[i]]
    pipe = make_pipeline(*steps)
    grid = GridSearchCV(pipe,param_grids[i]).fit(X_train, y_train)
    grids.append(grid)
```

Dette er kanskje litt lite ryddig, og det blir litt ekstraarbeid siden den kjører preprocess steps på nytt... Kan prøve å få alt inn i et enkelt GridSearch objekt, men da blir det kanskje litt vanskelig å se på .cv_results_ siden det er ulik parametergrid for ulike predictors ...?

Kan også bruke param_grid som er liste med dictionary... én for hver pipe.. men alt inn i samme Pipeline objekt... hmhmhm

```python 
pipe = Pipeline([('preprocessing', StandardScaler()), ('classifier', SVC())])

param_grid = [
    {'classifier': [SVC()],
     'preprocessing': [StandardScaler(), None],
     'classifier__gamma': [0.001, 0.01, 0.1, 1, 10, 100],
     'classifier__C': [0.001, 0.01, 0.1, 1, 10, 100]},
    {'classifier': [RandomForestClassifier(n_estimators=100)],
     'preprocessing': [None],
     'classifier__max_features': [1, 2, 3]}]

grid = GridSearchCV(pipe, param_grid, cv=5)
grid.fit(X_train, y_train)
```

### Linear Model

### Neighbours

### SVM

### Tree

### Discriminant analysis

### Ensemble

Boosting og bagging.. bedre enn enkelt modell. Redusere varians eller redusere bias.

```python
rf = RandomForestClassifier(n_estimators).fit(X,y) # vet ikke behov for andre å tune andre parametre..

feature_importance = rf.feature_importance_ # har noe med reduksjon i impurity for split på feature ...
```

### Dummy

Bruker til å lage baseline modell. Sammenligningsgrunnlag for mer kompliserte modeller.

```python
DummyClassifier(strategy=
                'stratified', # tilfeldig ut fra andel i training se
                'most_frequent', # alltid predikere majoritet
                'uniform', # tilfeldig
                'constant' # alltid predikere angitt verdi
               )
```

### Mixture

```python
gmm = GaussianMixture(n_components, 
                      covariance_type # full, tied, diag, spherical.. spesifisere om de har komponent til felles
                      )
# Methods
gmm.bic(X) # BIC på sample som fittet modellen på..

# pdf i én dimensjona
grid = np.linspace(-6,6,1000)
logprob = gmm.score_samples(grid[:,None])
pdf = np.exp(logprob)

# pdf i to dimensjoner
xx1, xx2 = np.mgrid[-1.5:2.5:0.01,-1:1.5:0.01]
xx = np.array([xx1.ravel(), xx2.ravel()]).T
pdf = np.exp(gmm.score_samples(xx)).reshape(xx1.shape)
```

### Cluster

Metoder for å gruppere observasjoner. Vil dele de inn i grupper slik at observasjonene er mest mulig like innad i grupper, samtidig som det er avstand mellom gruppene. Vi trenger da et mål på avstand mellom observasjoner. Et vanlig valg er eukledisk avstand mellom observasjonsvektorer, og det er da viktig at de har standardisert måleenhet slik at avstanden er meningsfull.

Tror vi kan bruke det til å lære mer om fordelingen i populasjonen. Kan tenke oss at det er en uobservert variabel assosiert med hver observasjon som indikerer slags gruppetilhørighet som vi forsøker å gjette oss til fra realiserte data... Vi kan deretter bruke dette til å beskrive egenskaper til de betingede fordelingene til hver av de konstruerte gruppene i populasjonen vår.

Har ulike algoritmer som gjør ulike antagelser om gruppene i den datagenerende prosessen. Tror jeg i hovedssak ser på
1. K-means: Minimerer avstand til center. Antar dermed at cluster er spredt i sirkel rundt hvert sentrum
2. Gaussian mixture: Har i tillegg kovariansmatrise som lærer fra data, åpner for ulik form. Dessuten soft boundary; har estimat på sannsynlighet for at ny observasjon tilhører hver av gruppene
3. DBSCAN (Density based spatital clustering with added noise): hm

```python
from sklearn.cluster import KMeans, DBSCAN
kmeans = KMeans(n_clusters).fit(X)
labels = kmeans.labels_
centers = kmeans.centers_

# albuemetode for å bestemme antall clusters. Vet ikke om kan utvide til andre algoritmer. trenger i så fall en metric
inertias = []
k_grid = range(1,6+1)
for k in k_grid:
    inertias.append(KMeans(k).fit(X).inertia_)

plt.plot(k_grid, inertias)
```

### Manifold

### Annet

Visualisere estimert sannsynlig for positiv klasse i binær klassifisering
```python
treshold = 0.5 # kan velge cut-off for decision boundary. default er 0.5
x0_min, x0_max = min(X[:, 0])-.2, max(X[:, 0])+.2
x1_min, x1_max = min(X[:, 1])-.2, max(X[:, 1])+.2
xx0, xx1 = np.mgrid[x0_min:x0_max:.01, x1_min:x1_max:.01]

clf = clf.fit(X,y)
yy = clf.predict_proba(np.c_[xx0.ravel(), xx1.ravel()])[:,1] # sannsynlig for positiv klasse
yy = yy.reshape(xx0.shape)

ax.contourf(xx0, xx1, yy, cmap='RdBu_r')
ax.contour(xx0,xx1,yy,levels=[treshold], linestyles='dashed', linewidths=3, colors='black')
ax.scatter(X[:, 0], X[:, 1], c=y, cmap='RdBu_r')
```

### Prosjekt workflow

Skriver ned midlertidig for å internalisere.

#### Big picture

Må ha veldefinert målsetning. Ha forståelse for hvordan maskinlæringsalgoritmen blir plassert i kontekst... hva er bruksområde. I praksis så er det ofte en del av en pipeline det output blir input i beslutningsprosses nedstrøms. Må skape verdi for nedstrømsbrukere. Utforming av prosjekt og valg av algoritme avhenger til dels av deres behov (høyest mulig presisjon eller transparens/tractability).

Hvorvidt prosjektet skaper verdi avhenger av forskjell til baseline/status quo. Se i hvilken grad vi øker presisjon og/eller reduserer kostnader.

#### Workspace

Vi begynner med å lage et workspace for prosjektet (ser her bort fra samarbeid/ekstern repo). Her kommer command line inn i bildet. Vil lage en root folder (?) og initialisere repo for versjonskontroll. Vil også lage et isolert virtuelt miljø for prosjektet som definerer hvilke _dependencies_ (eksterne kodebaser) den har tilgang til og i hvilke versjon. Ved å isolere det fra det globale miljøet som min partikulære pc har på det gitte tidspunktet så unngår jeg problem med at andre (med andre miljø) ikke kan kjøre det, og unngår også bugs som følge av endringer i ekstern kodebase.

Vil også organisere prosjektet på en god måte slik at andre kan samarbeide og replikere arbeidet mitt. Gode rutiner er viktig og dette må jeg vel utvikle etter hvert. Prosjektet er iterativ prosess med utforskning og prøving/feiling, men vil lagre ryddig versjon. Lagre kode i moduler og ha et main script som abstaherer fra implementasjonsdetaljer. Unngå notebooks til annet enn ren presentasjon.

#### Laste data

Vil unngå å manuelt laste ned data på min lokale masking og laste inn fra path. Det er poeng at det skal være enkelt for andre å bruke det samme scriptet til å oppnå samme resultat på sin pc. 

I praksis ligger data i relasjonell database og jeg må gjøre litt sql ting... litt usikker på om jeg kan gjøre dette fra python script eller om det blir separat del av workflow.

Alternativ kan jeg laste data fra nettet. Kan være litt utfordring med at store datafiler gjerne er komprimert, så må automatisere unzippingen fra python scriptet..

#### Exploratory data analysis

Må først bli litt kjent med data. Om mulig: liste kolonner, litt beskrivelse om hva de måler inkl målenhet og datatype. Vil ha oversikt over manglende observasjoner, univariate fordeling (sentraltendens, spredning, outliars) og litt om relasjon mellom variabler (korrelasjon som sammendragsmål).

Bruker pandas og seaborn til dette.

#### Modellering

Jeg inkluderer feature engineering som del av modellering. Hvor mye algoritme kan lære avhenger av hvordan vi representerer data. Spesielt viktig hvis vi har relativt få observasjon; hvis uendelig data så oppdager den hva som er viktig og hva som er trash. Kan bruke domanin knowledge til å konstrure variabler som er informativt for output (feriedager viktig for reise mm.)

Kan til dels bruke teori/domain knowledge til å bestemme hvilke variabler som er relevant eller ikke. Men dette kan til dels testes/automatiseres, så god idé å gjøre dette. Kan blant annet lage nye variabler som er kombinasjon av eksisterende (interaksjoner) mm. Vi operasjonaliserer dette gjennom transformers i sklearn med hyperparametre som vi kan søke over (eks: parameter som indikerer om vi inkluderer interaksjoner eller ikke).

Må dessuten skalere før vi sender inn i algoritme.

#### Presentasjon av prosjekt

Hva oppnådde vi? Resultat i forhold til baseline. Hva kan vi lære fra modellen? Hvilke inputs er viktige, noe om retning på effekten... gir mer kredibilitet dersom vi forstår $h$ vi har estimert. Performance på ulike segment av testdata.. kan vi si noe om systematiske svakheter/styrke.. forstå hvorfor den fungerer såpass bra som den gjør og ikke bedre.

Dernest kan vi si noe om prosessen. Hva som fungerte og hva som ikke gjorde det og noe om veien videre. Mulighet for bedre data?

#### Produksjon

Lagrer modell med ... hva? Litt usikker. Plassere i pipeline slik at får data inn og spytter ut output. Kan være at den mottar queries fra webside der bruker plugger inn input, hva vet vel jeg... tror ikke dette kan være mitt ansvar. for da være grenser. 

Kan også være greit å overvåke input og output for å se at ting er kosher og at modellen oppfører seg som den skal.